Assumoption:
- A1: Have model
- A2: Have all images arr loaded from transformed

Steps:
- S1: Split trian and test
- S2: Remove classfication layer (last 2 layer)
- S3: Predict all test images (result as encoded array) aka feature vectors 
- S4: Build KDTree index
- S5: find similar images

#### Deprecated

In [2]:
# def get_existed_df() -> pd.DataFrame:
#     parquet_file = pq.ParquetFile('./data/img.parquet')
#     df = None
    
#     for batch in parquet_file.iter_batches(batch_size=1000):
#         batch_df = batch.to_pandas()
#         df = pd.concat([batch_df, df]) if df is not None else batch_df
        
#         print(len(batch_df), len(df))

#     return df



# persist_path = './data/img.parquet'
# if os.path.exists(persist_path):
#     df_sim = get_existed_df()
# else:
#     df_sim = get_df_manually()
#     df_sim.to_parquet(persist_path)


## Dependencies

In [6]:
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import os
import cv2
import tensorflow.image as imtf
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import mahotas.features.texture as texture
import pyarrow
import pyarrow.parquet as pq

tqdm.pandas()

## Read Data

In [4]:
# Load Dataset
data_file_path = "./data/transforms.parquet"
df = pd.read_parquet(data_file_path)

# Note: Numpy array is treated as string here 
df['path'] = df['path'].str.replace('Furniture_Data','Transformed_Furniture_Data')
df.head()

,path,furniture,style
0,./Transformed_Transformed_Furniture_Data/table...,table,eclectic
1,./Transformed_Transformed_Furniture_Data/table...,table,eclectic
2,./Transformed_Transformed_Furniture_Data/table...,table,eclectic
3,./Transformed_Transformed_Furniture_Data/table...,table,eclectic
4,./Transformed_Transformed_Furniture_Data/table...,table,eclectic


In [5]:
df = df[

(544986, 3)

In [ ]:
def get_image(img_path):
    img_arr = cv2.imread(img_path)
    
    # Downsize image for faster model training. BB
    # Normalize the image to scale 0-1 for faster training time and better performance
    return cv2.resize(img_arr, (50,50)) / 255.0 

# Get img arr 
X = df['path'].progress_apply(lambda path: get_image(path))
X = np.array(X.to_list())


In [12]:
# Define Label
def get_index(furniture):
    return furniture_indexes[furniture]

# Define furniture index. We'll use this data format as target for training 
furniture_indexes = {furniture:idx for idx, furniture in enumerate(df['furniture'].unique(), 1)}
print(furniture_indexes)

y = df['furniture'].apply(lambda furniture: get_index(furniture)).to_list()
y = np.array(y)


{'table': 1, 'sofa': 2, 'lamp': 3, 'chair': 4, 'dresser': 5, 'bed': 6}


### [Deprecated] Calculate similarity features for later similarity computation

In [25]:
bins = 88

def get_histogram(image: np.ndarray):
    histogram = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    histogram = cv2.normalize(histogram, histogram).flatten()
    return histogram 

def get_texture_feature(image: np.ndarray):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = texture.haralick(gray_image)
    return np.mean(glcm, axis=0)

def get_compactness(image: np.ndarray):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding
    thresholded = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Calculate compactness for each contour
    compactness_values = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        compactness = perimeter / np.sqrt(area) if area > 0 else 0
        compactness_values.append(compactness)
    
    # Maximum Compactness: Choose the contour with the highest compactness value. 
    # This approach assumes that the object with the highest compactness is the most significant or relevant in the image.
    max_compactness_index = np.argmax(compactness_values)
    max_compactness_value = compactness_values[max_compactness_index]
    
    return max_compactness_value


# def get_similarity_attrs(row: pd.Series) -> pd.Series: 
#     print(row['path'])
#     image = cv2.imread(row['path'])

#     row['histogram'] = get_histogram(image)
#     row['texture_feature'] = get_texture_feature(image)
#     row['compactness'] = get_compactness(image)

#     return row 

# def get_similarity_attrs(path):
#     image = cv2.imread(path)
def get_similarity_attrs(image):
    histogram = get_histogram(image)
    texture_feature = get_texture_feature(image)
    compactness = get_compactness(image)
    return pd.Series([histogram, texture_feature, compactness], index=['histogram', 'texture_feature', 'compactness'])

In [ ]:
# Create new columns to store the results (take abt 1 hour to complete)
# df_sim[['histogram', 'texture_feature', 'compactness']] = df_sim['path'][:20].progress_apply(get_similarity_attrs)
df_test = df_sim['img_arr'][:20].progress_apply(get_similarity_attrs)

In [ ]:
# !pip install git+https://github.com/openai/CLIP.git
# !pip install open_clip_torch
# !pip install sentence_transformers

# import torch
# import open_clip
# import cv2
# from sentence_transformers import util
# from PIL import Image

# image processing model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(image1, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(image2, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score
print(f"similarity Score: ", round(generateScore(image1, image2), 2))
#similarity Score: 76.77

## Training

In [20]:
X.shape

(90082, 50, 50, 3)

In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
# Using stratify on y to ensure better distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train = np.squeeze(X_train)
y_train = np.squeeze(y_train)
X_test = np.squeeze(X_test)
y_test = np.squeeze(y_test)

In [9]:
print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")

X_train shape (109474, 50, 50, 3)
X_test shape (72983, 50, 50, 3)


In [10]:
print(f"y_train shape {y_train.shape}")
print(f"y_test shape {y_test.shape}")

y_train shape (109474,)
y_test shape (72983,)


In [11]:
from tensorflow.keras import layers, models, regularizers

# Create a sequential model
model = models.Sequential()

# Add convolutional layers
# model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(224, 224,3),kernel_regularizer=regularizers.l2(0.001)))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(50, 50,3), padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(256, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(512, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(512*2*2, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))



# Add a dense layer
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.2)) # add a dropout layer with dropout rate of 0.2


# Add the output layer
model.add(layers.Dense(64, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Epoch 1/5


2024-04-30 13:58:25.130468: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3422/3422 [==============================] - 81s 24ms/step - loss: 0.5017 - accuracy: 0.8230 - val_loss: 0.3039 - val_accuracy: 0.8983
Epoch 2/5
3422/3422 [==============================] - 74s 22ms/step - loss: 0.2658 - accuracy: 0.9104 - val_loss: 0.2659 - val_accuracy: 0.9150
Epoch 3/5
3422/3422 [==============================] - 73s 21ms/step - loss: 0.2088 - accuracy: 0.9304 - val_loss: 0.2210 - val_accuracy: 0.9261
Epoch 4/5
3422/3422 [==============================] - 66s 19ms/step - loss: 0.1738 - accuracy: 0.9420 - val_loss: 0.2074 - val_accuracy: 0.9320
Epoch 5/5
3422/3422 [==============================] - 66s 19ms/step - loss: 0.1496 - accuracy: 0.9489 - val_loss: 0.1728 - val_accuracy: 0.9444


### Predict

#### Load Model

In [7]:
# load model
from tensorflow.keras import models

model_dir = "./models"
model_h5_file = os.path.join(model_dir, "model_task_2_cnn_classification.h5")

model = models.load_model(model_h5_file)
model

In [64]:
# (Adhoc) Remove the last 2 layers for better encoding (according to ChatGPT) 
encoder = models.Sequential()
for layer in model.layers[:-2]:  # Exclude the last two layers (flatten and dense)
    encoder.add(layer)
encoder

Read raws images that we'll use as reference for similairty search

In [22]:
data_file_path = "./data/raws.parquet"
df_sample = pd.read_parquet(data_file_path)


furnitures = df_sample['furniture'].unique()
arr_paths = []
arr_imgs = []

for fur in furnitures:
    df_fur = df_sample.query(f"furniture == '{fur}'")[:500]
    
    arr_img_fur = df_fur['path'].progress_apply(lambda path: get_image(path))
    
    arr_paths.extend(df_fur['path'].to_list())
    arr_imgs.extend(arr_img_fur)


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Use the encoder to encode the image for better vector search

In [19]:
feature_vectors = [encoder.predict(np.expand_dims(img, axis=0)).flatten()  for img in tqdm(arr_imgs)]
feature_vectors =  np.array(feature_vectors)

feature_vectors


  0%|          | 0/3000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 12ms/step


2024-05-05 16:05:23.511811: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 12ms/step


array([[0.        , 2.8793952 , 1.5406963 , ..., 0.        , 0.9400946 ,
        0.        ],
       [0.        , 6.2194047 , 1.7200975 , ..., 0.        , 1.7547543 ,
        0.        ],
       [0.        , 2.541437  , 0.        , ..., 0.        , 0.04728572,
        0.        ],
       ...,
       [0.        , 5.582647  , 0.        , ..., 0.        , 0.7491129 ,
        0.9001496 ],
       [0.        , 3.4339526 , 2.4483483 , ..., 0.        , 0.        ,
        0.59078324],
       [0.        , 0.88045555, 0.41729409, ..., 0.24426728, 0.        ,
        0.40553612]], dtype=float32)

Load K Neighbors Model to similarity search

In [66]:
from sklearn.neighbors import KDTree
# Build KDTree index
kdtree_index = KDTree(feature_vectors)

# Function to find similar images for a given query image
def find_similar_images(query_image_path, top_k=5):
    # Preprocess query image
    query_img = get_image(query_image_path)
    # Extract features from query image using your model
    query_features = encoder.predict(np.expand_dims(query_img, axis=0)).flatten()  # Flatten feature vector
    
    # Perform nearest neighbor search using KDTree index
    distances, indices = kdtree_index.query(query_features.reshape(1, -1), k=top_k)

    similarity_scores = 1 / (1 + distances) # (unused for now)
    
    # Return paths of similar images
    similar_image_paths = [arr_paths[idx] for idx in indices[0]]
    return similar_image_paths



Input a random image and check for similar images

In [67]:
input_img_path = './Furniture_Data/tables/Midcentury/43697midcentury-side-tables-and-end-tables.jpg'
find_similar_images(input_img_path)

1/1 [==============================] - 0s 12ms/step


['./Furniture_Data/tables/Midcentury/43697midcentury-side-tables-and-end-tables.jpg',
 './Furniture_Data/tables/Eclectic/4943eclectic-table.jpg',
 './Furniture_Data/tables/Eclectic/4051eclectic-side-tables-and-end-tables.jpg',
 './Furniture_Data/tables/Eclectic/3781eclectic-side-tables-and-end-tables.jpg',
 './Furniture_Data/tables/Eclectic/3784eclectic-side-tables-and-end-tables.jpg']

## Post execution

Save new dataset curated for similarity score calculation

In [ ]:
# Define the directory to save the model
def save_dataset(df: pd.DataFrame, data_file_name: str) -> None:
    data_dir = "./data"
    data_file_path = os.path.join(data_dir, data_file_name)
    
    # Check if the directory exists, and if not, create it
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    df.to_parquet(data_file_path, index=False, engine='pyarrow')
    print(f"Saved DataFrame to {data_file_path}")

data_file_name = "similarities.parquet"
save_dataset(dummy, data_file_name)

Save model 

In [12]:
model_dir = "./models"
model_h5_file = os.path.join(model_dir, "model_task_2_cnn_classification.h5")

# Check if the directory exists, and if not, create it
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model.save(model_h5_file)